# 1. Utilice la base de datos de California housing

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt

### Obtener los datos

In [10]:
housing = fetch_california_housing()

# 2. Realice la partición de entrenamiento, validación y prueba (train, val and test)

In [13]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

# 3. Realice la transformación de los datos (StandardScaler)

In [14]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.fit_transform(X_test)

# 4. Utilizando la librería de Keras crea un modelo MLP .

In [15]:
model = tf.keras.Sequential([tf.keras.layers.Dense(30, activation = 'relu', input_shape = X_train.shape[1:]),tf.keras.layers.Dense(1)])

In [16]:
model.compile(loss = tf.losses.mean_squared_error, optimizer = 'sgd')

In [17]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 1.5084 - val_loss: 2.5404
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 1.1082 - val_loss: 0.4807
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4171 - val_loss: 0.3991
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3840 - val_loss: 0.4060
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3688 - val_loss: 0.3868
Epoch 6/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3617 - val_loss: 0.4007
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3525 - val_loss: 0.3850
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3479 - val_loss: 0.3937
Epoch 9/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3434 - val_loss: 0.4073
Epoch 10/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3413 - val_loss: 0.4171

In [18]:
mse_test = model.evaluate(X_test, y_test)


162/162 [==============================] - 0s 1ms/step - loss: 0.4121


# 5. Utiliza la función de RandomizedSearchCV para encontrar los mejores parámetros de: capas ocultas, número de unidades, learning rate y dropout

In [29]:
def build_model(n_hidden = 1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr='learning_rate')
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [32]:
modelo_nuevo = build_model()
modelo_nuevo.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 30)                270       
                                                                 
 dense_15 (Dense)            (None, 1)                 31        
                                                                 
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


### Crear el Regresor basado en el modelo creado

In [31]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

C:\Users\Emmanuel\AppData\Local\Temp\ipykernel_2280\1709004121.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


### Buscar mejores parametros

In [40]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [41]:
param_distribs = {
    "n_hidden":[0,1,2,3],
    "n_neurons":np.arange(1,100),
    "learning_rate":reciprocal(3e-4, 3e-2),
}

### Entrenar el modelo con la busqueda de paremetros

In [43]:
rnd_serch_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_serch_cv.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), callbacks = [keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/20
242/242 [==============================] - 1s 2ms/step - loss: 1.0281 - val_loss: 0.7053
Epoch 2/20
242/242 [==============================] - 1s 3ms/step - loss: 0.5787 - val_loss: 0.4344
Epoch 3/20
242/242 [==============================] - 1s 2ms/step - loss: 0.4220 - val_loss: 0.3951
Epoch 4/20
242/242 [==============================] - 1s 2ms/step - loss: 0.3898 - val_loss: 0.3726
Epoch 5/20
242/242 [==============================] - 1s 2ms/step - loss: 0.3690 - val_loss: 0.3748
Epoch 6/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3567 - val_loss: 0.3773
Epoch 7/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3465 - val_loss: 0.3652
Epoch 8/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3403 - val_loss: 0.3831
Epoch 9/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3335 - val_loss: 0.3726
Epoch 10/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3302 - val_loss: 0.3761

Epoch 1/20
242/242 [==============================] - 1s 2ms/step - loss: 0.8471 - val_loss: 0.4818
Epoch 2/20
242/242 [==============================] - 0s 2ms/step - loss: 0.4677 - val_loss: 0.4104
Epoch 3/20
242/242 [==============================] - 0s 2ms/step - loss: 0.4218 - val_loss: 0.3926
Epoch 4/20
242/242 [==============================] - 0s 2ms/step - loss: 0.4028 - val_loss: 0.3843
Epoch 5/20
242/242 [==============================] - 1s 2ms/step - loss: 0.3876 - val_loss: 0.3858
Epoch 6/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3815 - val_loss: 0.3653
Epoch 7/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3682 - val_loss: 0.3758
Epoch 8/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3609 - val_loss: 0.3678
Epoch 9/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3508 - val_loss: 0.4174
Epoch 10/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3457 - val_loss: 0.4235

Epoch 1/20
242/242 [==============================] - 1s 2ms/step - loss: 0.8114 - val_loss: 0.4982
Epoch 2/20
242/242 [==============================] - 0s 2ms/step - loss: 0.4815 - val_loss: 0.4136
Epoch 3/20
242/242 [==============================] - 0s 2ms/step - loss: 0.4021 - val_loss: 0.3823
Epoch 4/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3798 - val_loss: 0.3764
Epoch 5/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3686 - val_loss: 0.3745
Epoch 6/20
242/242 [==============================] - 1s 3ms/step - loss: 0.3569 - val_loss: 0.3970
Epoch 7/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3510 - val_loss: 0.3642
Epoch 8/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3451 - val_loss: 0.3535
Epoch 9/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3379 - val_loss: 0.3513
Epoch 10/20
242/242 [==============================] - 1s 2ms/step - loss: 0.3325 - val_loss: 0.3609

Epoch 1/20
242/242 [==============================] - 2s 5ms/step - loss: 3.3541 - val_loss: 9.7855
Epoch 2/20
242/242 [==============================] - 0s 2ms/step - loss: 32.0574 - val_loss: 263.9477
Epoch 3/20
242/242 [==============================] - 0s 2ms/step - loss: 989.5757 - val_loss: 8040.5151
Epoch 4/20
242/242 [==============================] - 0s 2ms/step - loss: 27477.0508 - val_loss: 249642.5938
Epoch 5/20
242/242 [==============================] - 0s 2ms/step - loss: 974712.7500 - val_loss: 7760225.0000
Epoch 6/20
242/242 [==============================] - 0s 2ms/step - loss: 30052402.0000 - val_loss: 240604496.0000
Epoch 7/20
242/242 [==============================] - 0s 2ms/step - loss: 860565696.0000 - val_loss: 7476563968.0000
Epoch 8/20
242/242 [==============================] - 0s 2ms/step - loss: 27816163328.0000 - val_loss: 233479143424.0000
Epoch 9/20
242/242 [==============================] - 0s 2ms/step - loss: 1568082231296.0000 - val_loss: 7293288054784.

Epoch 1/20
242/242 [==============================] - 1s 2ms/step - loss: 1.3815 - val_loss: 0.8186
Epoch 2/20
242/242 [==============================] - 0s 2ms/step - loss: 0.6005 - val_loss: 0.6423
Epoch 3/20
242/242 [==============================] - 0s 2ms/step - loss: 0.5664 - val_loss: 0.5535
Epoch 4/20
242/242 [==============================] - 1s 2ms/step - loss: 0.5489 - val_loss: 0.5237
Epoch 5/20
242/242 [==============================] - 0s 2ms/step - loss: 0.5383 - val_loss: 0.5269
Epoch 6/20
242/242 [==============================] - 0s 2ms/step - loss: 0.5321 - val_loss: 0.5552
Epoch 7/20
242/242 [==============================] - 0s 2ms/step - loss: 0.5275 - val_loss: 0.5893
Epoch 8/20
242/242 [==============================] - 0s 2ms/step - loss: 0.5220 - val_loss: 0.6330
Epoch 9/20
242/242 [==============================] - 0s 2ms/step - loss: 0.5229 - val_loss: 0.6754
Epoch 10/20
242/242 [==============================] - 0s 2ms/step - loss: 0.5216 - val_loss: 0.7198

...

Epoch 1/20
242/242 [==============================] - 1s 2ms/step - loss: 3.1569 - val_loss: 1.2404
Epoch 2/20
242/242 [==============================] - 1s 2ms/step - loss: 0.4801 - val_loss: 0.7811
Epoch 3/20
242/242 [==============================] - 0s 2ms/step - loss: 0.4152 - val_loss: 0.7232
Epoch 4/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3982 - val_loss: 0.7474
Epoch 5/20
242/242 [==============================] - 1s 2ms/step - loss: 0.3846 - val_loss: 0.6927
Epoch 6/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3671 - val_loss: 0.7272
Epoch 7/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3629 - val_loss: 0.7351
Epoch 8/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3578 - val_loss: 0.6718
Epoch 9/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3534 - val_loss: 0.6425
Epoch 10/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3503 - val_loss: 0.6804

Epoch 1/20
242/242 [==============================] - 1s 2ms/step - loss: 0.9672 - val_loss: 0.5936
Epoch 2/20
242/242 [==============================] - 0s 2ms/step - loss: 0.5519 - val_loss: 0.4738
Epoch 3/20
242/242 [==============================] - 0s 2ms/step - loss: 0.4863 - val_loss: 0.4643
Epoch 4/20
242/242 [==============================] - 1s 2ms/step - loss: 0.4606 - val_loss: 0.4803
Epoch 5/20
242/242 [==============================] - 1s 2ms/step - loss: 0.4456 - val_loss: 0.4832
Epoch 6/20
242/242 [==============================] - 0s 2ms/step - loss: 0.4391 - val_loss: 0.4842
Epoch 7/20
242/242 [==============================] - 0s 2ms/step - loss: 0.4235 - val_loss: 0.4809
Epoch 8/20
242/242 [==============================] - 0s 2ms/step - loss: 0.4198 - val_loss: 0.4726
Epoch 9/20
242/242 [==============================] - 1s 2ms/step - loss: 0.4122 - val_loss: 0.4555
Epoch 10/20
242/242 [==============================] - 0s 2ms/step - loss: 0.4094 - val_loss: 0.4611

Epoch 1/20
242/242 [==============================] - 1s 2ms/step - loss: 1.8999 - val_loss: 0.5842
Epoch 2/20
242/242 [==============================] - 1s 2ms/step - loss: 0.4781 - val_loss: 0.4969
Epoch 3/20
242/242 [==============================] - 1s 2ms/step - loss: 0.4155 - val_loss: 0.4649
Epoch 4/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3908 - val_loss: 0.4546
Epoch 5/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3816 - val_loss: 0.4489
Epoch 6/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3713 - val_loss: 0.4597
Epoch 7/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3651 - val_loss: 0.4516
Epoch 8/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3616 - val_loss: 0.4719
Epoch 9/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3594 - val_loss: 0.4306
Epoch 10/20
242/242 [==============================] - 0s 2ms/step - loss: 0.3566 - val_loss: 0.4377

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.7153 - val_loss: 0.5051
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5564 - val_loss: 0.4517
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4544 - val_loss: 0.3853
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3860 - val_loss: 0.3734
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3685 - val_loss: 0.3731
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3566 - val_loss: 0.3699
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3469 - val_loss: 0.3681
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3409 - val_loss: 0.3877
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3334 - val_loss: 0.4474
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3282 - val_loss: 0.3911

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x0000025FFE8CAE20>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025FFE4E9910>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

### Obtener el conjunto de parámetros que tuvo el mejor rendimiento y su puntuación

In [44]:
rnd_serch_cv.best_params_

{'learning_rate': 0.0005331747900797502, 'n_hidden': 3, 'n_neurons': 65}

In [45]:
rnd_serch_cv.best_score_

-0.33153602480888367